In [22]:
import pandas as pd
import numpy as np
import datetime
from string import Template
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
def get_MTA_data(startdate, enddate):
    data = pd.DataFrame()
    startdate = pd.to_datetime(startdate)  ### Assumption: Dates are Saturdays
    enddate = pd.to_datetime(enddate)
    
   # url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_180623.txt'
    urlbase = Template('http://web.mta.info/developers/data/nyct/turnstile/turnstile_$date.txt')
    
    week = pd.Timedelta('7 days')
    next_saturday = startdate
    while(next_saturday <= enddate):
        urlday = next_saturday.strftime('%y%m%d')
        url = urlbase.substitute(date=urlday)
        next_saturday = next_saturday+week
        newdata = pd.read_csv(url)
        data = pd.concat([data,newdata])
        
        
   # url = urlbase.substitute(date=date)
    return(data)
    




In [4]:
data = get_MTA_data('04/28/2018','06/30/2018')


In [5]:
data.to_pickle("MTA_Apr_June.pkl")

In [5]:
data.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,00:00:00,REGULAR,6667150,2259901
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,04:00:00,REGULAR,6667173,2259909
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,08:00:00,REGULAR,6667189,2259938
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,12:00:00,REGULAR,6667305,2260006
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/23/2018,16:00:00,REGULAR,6667534,2260066


In [8]:
data.columns = [column.strip() for column in data.columns]

In [9]:
data['date_time'] = data['DATE']+' '+data['TIME']
data['date_time'] = pd.to_datetime(data['date_time'])

In [10]:
data.index = data['date_time']

#3: Time Intervals

Identifying unique turnstiles:

In [11]:
data['turnstile_id'] = data['STATION']+'_'+data['C/A']+'_'+data['UNIT']+'_'+data['SCP']

In [12]:
def difference(srs):
    return srs.max() - srs.min()

In [13]:
df_daily = data.groupby(by=['turnstile_id', 'DATE']).agg(difference)

In [14]:
by_station =  data.groupby(by=['STATION','turnstile_id', 'DATE']).agg(difference)

In [15]:
daily_totals = by_station.groupby(by=['STATION','DATE']).sum()

In [16]:

daily_totals.reset_index(inplace=True)


In [17]:
daily_totals["DATE"] = pd.to_datetime(daily_totals["DATE"])

In [19]:
daily_totals = daily_totals[daily_totals["ENTRIES"]<200000]

In [81]:

daily_totals.index = daily_totals["DATE"]


daily_totals_by_week = daily_totals.groupby("STATION").resample('W', how='sum').reset_index()

C:\Users\Aditya\Anaconda3\lib\site-packages\pandas\core\groupby.py:1324: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  return get_resampler_for_grouping(self, rule, *args, **kwargs)


In [82]:
daily_totals_by_week.head()

,STATION,DATE,ENTRIES,EXITS
0,1 AV,2018-04-22,28487,32063
1,1 AV,2018-04-29,122962,138641
2,1 AV,2018-05-06,122329,137672
3,1 AV,2018-05-13,118740,133792
4,1 AV,2018-05-20,123731,141572


In [85]:
weekly_totals = daily_totals_by_week.groupby("STATION").mean().reset_index()
weekly_totals["mean_percentage"] = (weekly_totals["ENTRIES"]/weekly_totals["ENTRIES"].sum())*len(weekly_totals)

In [86]:
weekly_totals.sort_values("ENTRIES",ascending=False).head()

,STATION,ENTRIES,EXITS,mean_percentage
60,34 ST-PENN STA,784022.727273,678365.545455,10.041845
232,GRD CNTRL-42 ST,720653.636364,690145.181818,9.230207
58,34 ST-HERALD SQ,594431.181818,560372.909091,7.613536
45,23 ST,546372.636364,424825.909091,6.997997
14,14 ST-UNION SQ,521808.545455,488234.545455,6.683378
